In [2]:
import torch
print(torch.cuda.device_count())  # should show the number of available GPUs

1


In [ ]:
#!pip install torch
#!pip install accelerate datasets transformers evaluate
import torch, transformers, datasets, accelerate, evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from accelerate.utils.memory import clear_device_cache
from datasets import get_dataset_split_names, load_dataset, get_dataset_config_names

h:\GITHUB REPOS\AI-Practises\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full")

In [3]:
get_dataset_split_names("yelp_review_full")

['train', 'test']

In [4]:
dataset["train"].features # features of the 'train' dataset

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars']),
 'text': Value('string')}

In [5]:
dataset["train"][17] # a random example from the 'train' dataset

{'label': 4,
 'text': 'Old school.....traditional \\"mom \'n pop\\" quality and perfection. The best fish and chips you\'ll ever enjoy and equally superb fried shrimp. A great out of the way, non-corporate, vestige of Americana. You will love it.'}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#### Create small subsets of the data using only 1000 examples for train and eval
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2124.67 examples/s]


In [10]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
training_args = TrainingArguments(
    report_to="none",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10,
    output_dir="test_trainer",
    eval_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
training_args = TrainingArguments(
    report_to="none",
    output_dir="test_trainer",
    eval_strategy="epoch",
    num_train_epochs=5 ,
    learning_rate=1e-3, ### made learning rate bigger
    per_device_train_batch_size=8,  ### smaller batch size
    per_device_eval_batch_size=8,
    weight_decay=0.1,  ### 10x bigger weight decay (regularizer)
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,  ### smaller gradient accum steps
    ### cf smaller batch size also
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.690600,1.851357,0.166000
2,1.772600,1.654051,0.205000
3,1.747500,1.695082,0.205000
4,1.648500,1.644969,0.205000
5,1.636700,1.609953,0.220000


TrainOutput(global_step=625, training_loss=1.693303709411621, metrics={'train_runtime': 640.4359, 'train_samples_per_second': 7.807, 'train_steps_per_second': 0.976, 'total_flos': 1315590712320000.0, 'train_loss': 1.693303709411621, 'epoch': 5.0})